<a href="https://colab.research.google.com/github/kikiru328/enterprise_analysis/blob/main/%EA%B1%B0%EC%A3%BC%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', my_path)
sys.path.insert(0,my_path)

# 한글폰트
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
## 돌리고 런타임 재시작
## 드라이브 마운트, 한글폰트설치코드 제외 셀 실행

Mounted at /content/drive
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 3s (3,146 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading datab

생각해본 거주지 범주화 기준
1. 거주지별 연령대 비중과 총매출액을 기준
고령(60) / 중장년층(4-50) / 청년(2-30) / 청소년(10)
거주지별 비율 구해서
총 매출액에서 고령이 쓴 매출액 비중
총 매출액에서 중장년층이 쓴 매출액 비중
총 매출액에서 청년이 쓴 매출액 비중
총 매출액에서 청소년이 쓴 매출액 비중
으로 나눠서 가장 높은 연령대로 범주화 (제대로 범주화가 되지 않을 가능성 높음)

In [1]:
#  Directory 변경 (모듈이용)
%pwd
%cd '../content/notebooks'

# module import
## Data Load
from pickle5 import pickle 

## Basic module
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')


/content/drive/My Drive/Colab Notebooks


In [7]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/data/EDA/기존_purprd_cust.pkl','rb') as file:
  df = pickle.load(file)

In [8]:
df

,성별,연령대,거주지역,제휴사,멤버십가입개수,영수증번호,고객번호,점포코드,구매일자,구매년도,구매월,구매시간,대분류명,중분류명,소분류명,구매금액
0,F,50대,경기 김포시,A,0,2757986,15148,10,2014-09-20,2014,9,20,가구/인테리어,일용잡화,위생세제,1000
1,F,50대,경기 김포시,A,0,2757986,15148,10,2014-09-20,2014,9,20,가공식품,가공식품,일반가공식품,4100
2,F,60세이상,경기 김포시,A,0,2757987,400,10,2014-09-20,2014,9,12,신선식품,농산물,채소,5000
3,F,60세이상,경기 김포시,A,0,2757987,400,10,2014-09-20,2014,9,12,가공식품,젓갈/반찬,벌크김치,10200
4,F,50대,경기 김포시,A,0,2757988,2043,10,2014-09-20,2014,9,13,가구/인테리어,일용잡화,위생세제,3950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593025,M,30대,경기 김포시,D,1,253839,15835,40,2015-10-26,2015,10,13,가공식품,음료,일반차음료,3000
28593026,M,30대,경기 김포시,D,1,253839,15835,40,2015-10-26,2015,10,13,가공식품,음료,커피음료,1300
28593027,M,50대,울산 동구,D,0,253840,2070,40,2015-10-26,2015,10,16,일상용품,헤어케어,샴푸,59700
28593028,M,30대,경기 김포시,D,1,253841,15835,40,2015-10-26,2015,10,23,가공식품,과자,하드캔디,2000


In [20]:
test = df[['고객번호','연령대','거주지역','구매금액']]
test

,고객번호,연령대,거주지역,구매금액
0,15148,50대,경기 김포시,1000
1,15148,50대,경기 김포시,4100
2,400,60세이상,경기 김포시,5000
3,400,60세이상,경기 김포시,10200
4,2043,50대,경기 김포시,3950
...,...,...,...,...
28593025,15835,30대,경기 김포시,3000
28593026,15835,30대,경기 김포시,1300
28593027,2070,50대,울산 동구,59700
28593028,15835,30대,경기 김포시,2000


In [44]:
test_group = test.groupby('고객번호')[['연령대']].count()

In [ ]:
test_group = test_group.reset_index()
test_group = test_group.rename(columns={'연령대':'구매횟수'})
test_group

In [ ]:
me_test = test.groupby(['고객번호','연령대']).count()
me_test = me_test.reset_index()
me_test

In [91]:
merged_test = pd.merge(test_group,me_test[['고객번호','연령대']],on='고객번호',how='left')
merged_test = merged_test[['고객번호','연령대','구매횟수']]
merged_test['고객번호'].nunique()

19085

In [116]:
merged_test

,고객번호,연령대,구매횟수
0,1,60세이상,1057
1,2,60세이상,1116
2,3,60세이상,1191
3,4,60세이상,930
4,5,60세이상,719
...,...,...,...
19080,19356,40대,1278
19081,19357,40대,1232
19082,19359,50대,1332
19083,19360,40대,544


In [98]:
A = test[['거주지역','구매금액','고객번호']]
A = A.groupby('고객번호')[['거주지역','구매금액']].agg({'거주지역':'first','구매금액':'sum'})
A = A.reset_index()
print(A['고객번호'].nunique())
A

19085


,고객번호,거주지역,구매금액
0,1,서울 강남구,85042550
1,2,경기 김포시,99880760
2,3,서울 은평구,3165167
3,4,서울 노원구,16693810
4,5,경기 김포시,13237070
...,...,...,...
19080,19356,경기 김포시,6245805
19081,19357,대전 유성구,8664915
19082,19359,경기 김포시,3259137
19083,19360,서울 강서구,4726290


In [182]:
merged_test2 = pd.merge(merged_test,A, on='고객번호',how='left')
main = merged_test2.copy()

In [183]:
main
main['연령대수'] = 1
main
t = main.groupby(['거주지역','연령대'])[['구매횟수','구매금액','연령대수']].agg({'구매금액':'sum','구매횟수':'sum','연령대수':'sum'})
t = t.reset_index()
t

,거주지역,연령대,구매금액,구매횟수,연령대수
0,강원도 철원,20대,128211963,17156,13
1,강원도 철원,30대,642281027,80572,44
2,강원도 철원,40대,1254422526,148137,74
3,강원도 철원,50대,549747765,74010,42
4,강원도 철원,60세이상,74368424,8155,4
...,...,...,...,...,...
204,충청북도,20대,208734492,16229,15
205,충청북도,30대,1335732623,150460,74
206,충청북도,40대,1963487959,286482,148
207,충청북도,50대,1376937432,145959,88


In [184]:
do_Li= list(t['거주지역'].unique())
print(t['거주지역'].nunique())
len(do_Li)

40


40

In [185]:
t = t.drop('연령대_비율',axis=1)
t

KeyError: ignored

In [205]:
a = pd.DataFrame()
for i in do_Li:
  i= t[t['거주지역']==i]
  i['연령대비율'] = i['연령대수'] / i['연령대수'].sum()
  i['total'] = i['연령대비율'].sum()
  a = pd.concat([a,i])
display(a)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,거주지역,연령대,구매금액,구매횟수,연령대수,연령대비율,total
0,강원도 철원,20대,128211963,17156,13,0.073446,1.0
1,강원도 철원,30대,642281027,80572,44,0.248588,1.0
2,강원도 철원,40대,1254422526,148137,74,0.418079,1.0
3,강원도 철원,50대,549747765,74010,42,0.237288,1.0
4,강원도 철원,60세이상,74368424,8155,4,0.022599,1.0
...,...,...,...,...,...,...,...
204,충청북도,20대,208734492,16229,15,0.044379,1.0
205,충청북도,30대,1335732623,150460,74,0.218935,1.0
206,충청북도,40대,1963487959,286482,148,0.437870,1.0
207,충청북도,50대,1376937432,145959,88,0.260355,1.0


In [211]:
#검증
B = a.groupby('거주지역')[['연령대','total']].agg({'연령대':'count','total':'sum'})
B.head(8)

,연령대,total
거주지역,,
강원도 철원,5,5.0
경기 김포시,6,6.0
경상남도 함양,6,6.0
경상북도 영주,5,5.0
광주 북구,6,6.0
대구 동구,5,5.0
대전 유성구,5,5.0
부산 기장,6,6.0


In [212]:
a

,거주지역,연령대,구매금액,구매횟수,연령대수,연령대비율,total
0,강원도 철원,20대,128211963,17156,13,0.073446,1.0
1,강원도 철원,30대,642281027,80572,44,0.248588,1.0
2,강원도 철원,40대,1254422526,148137,74,0.418079,1.0
3,강원도 철원,50대,549747765,74010,42,0.237288,1.0
4,강원도 철원,60세이상,74368424,8155,4,0.022599,1.0
...,...,...,...,...,...,...,...
204,충청북도,20대,208734492,16229,15,0.044379,1.0
205,충청북도,30대,1335732623,150460,74,0.218935,1.0
206,충청북도,40대,1963487959,286482,148,0.437870,1.0
207,충청북도,50대,1376937432,145959,88,0.260355,1.0


In [215]:
b = pd.DataFrame()
for i in do_Li:
  i= a[a['거주지역']==i]
  i['구매금액비율'] = i['구매금액'] / i['구매금액'].sum()
  i['구매금액비율_total'] = i['구매금액비율'].sum()
  b = pd.concat([b,i])
display(b)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,거주지역,연령대,구매금액,구매횟수,연령대수,연령대비율,total,구매금액비율,구매금액비율_total
0,강원도 철원,20대,128211963,17156,13,0.073446,1.0,0.048400,1.0
1,강원도 철원,30대,642281027,80572,44,0.248588,1.0,0.242459,1.0
2,강원도 철원,40대,1254422526,148137,74,0.418079,1.0,0.473540,1.0
3,강원도 철원,50대,549747765,74010,42,0.237288,1.0,0.207528,1.0
4,강원도 철원,60세이상,74368424,8155,4,0.022599,1.0,0.028074,1.0
...,...,...,...,...,...,...,...,...,...
204,충청북도,20대,208734492,16229,15,0.044379,1.0,0.041186,1.0
205,충청북도,30대,1335732623,150460,74,0.218935,1.0,0.263560,1.0
206,충청북도,40대,1963487959,286482,148,0.437870,1.0,0.387426,1.0
207,충청북도,50대,1376937432,145959,88,0.260355,1.0,0.271691,1.0


In [217]:
b[b['거주지역']=='충청북도']['구매금액비율'].sum()

1.0

In [219]:
b = b[['거주지역','연령대','연령대수','연령대비율','total','구매금액','구매횟수','구매금액비율','구매금액비율_total']]
b

,거주지역,연령대,연령대수,연령대비율,total,구매금액,구매횟수,구매금액비율,구매금액비율_total
0,강원도 철원,20대,13,0.073446,1.0,128211963,17156,0.048400,1.0
1,강원도 철원,30대,44,0.248588,1.0,642281027,80572,0.242459,1.0
2,강원도 철원,40대,74,0.418079,1.0,1254422526,148137,0.473540,1.0
3,강원도 철원,50대,42,0.237288,1.0,549747765,74010,0.207528,1.0
4,강원도 철원,60세이상,4,0.022599,1.0,74368424,8155,0.028074,1.0
...,...,...,...,...,...,...,...,...,...
204,충청북도,20대,15,0.044379,1.0,208734492,16229,0.041186,1.0
205,충청북도,30대,74,0.218935,1.0,1335732623,150460,0.263560,1.0
206,충청북도,40대,148,0.437870,1.0,1963487959,286482,0.387426,1.0
207,충청북도,50대,88,0.260355,1.0,1376937432,145959,0.271691,1.0


In [220]:
with open('/content/drive/MyDrive/공유문서/1차 프로젝트/5조/매출감소고객_예측모델_변수생성/고객속성변수/거주지연령구매비율.pkl','wb') as file:
  pickle.dump(b,file)

In [240]:
b

,거주지역,연령대,연령대수,연령대비율,total,구매금액,구매횟수,구매금액비율,구매금액비율_total
0,강원도 철원,20대,13,0.073446,1.0,128211963,17156,0.048400,1.0
1,강원도 철원,30대,44,0.248588,1.0,642281027,80572,0.242459,1.0
2,강원도 철원,40대,74,0.418079,1.0,1254422526,148137,0.473540,1.0
3,강원도 철원,50대,42,0.237288,1.0,549747765,74010,0.207528,1.0
4,강원도 철원,60세이상,4,0.022599,1.0,74368424,8155,0.028074,1.0
...,...,...,...,...,...,...,...,...,...
204,충청북도,20대,15,0.044379,1.0,208734492,16229,0.041186,1.0
205,충청북도,30대,74,0.218935,1.0,1335732623,150460,0.263560,1.0
206,충청북도,40대,148,0.437870,1.0,1963487959,286482,0.387426,1.0
207,충청북도,50대,88,0.260355,1.0,1376937432,145959,0.271691,1.0


In [238]:
x

,거주지역,연령대,연령대수,연령대비율,total,구매금액,구매횟수,구매금액비율,구매금액비율_total
19,경상북도 영주,40대,162,0.469565,1.0,4265789338,293950,0.381351,1.0
14,경상남도 함양,40대,389,0.459811,1.0,17519609522,580128,0.470389,1.0
93,서울 도봉구,40대,99,0.452055,1.0,3808950279,123246,0.439343,1.0
47,서울 강남구,40대,264,0.447458,1.0,14236107588,349630,0.460058,1.0
196,전라북도 군산,40대,246,0.443243,1.0,6799425151,380701,0.409021,1.0
...,...,...,...,...,...,...,...,...,...
22,광주 북구,19세이하,1,0.001585,1.0,14323429,1815,0.000699,1.0
11,경상남도 함양,19세이하,1,0.001182,1.0,17680390,337,0.000475,1.0
131,서울 송파구,19세이하,1,0.000706,1.0,2174180,214,0.000034,1.0
5,경기 김포시,19세이하,4,0.000676,1.0,23190889,5963,0.000122,1.0
